In [ ]:
import transformers

In [ ]:
# Hugging face trainer to build classification model for text data

In [ ]:
#Dataset class
#Model
#Trainer - training arguments

In [ ]:
pip install torch torchvision torchaudio transformers datasets

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import get_scheduler
from tqdm.auto import tqdm


# **Loading and Splitting the IMDB dataset**

In [ ]:
dataset = load_dataset("imdb")

In [ ]:
# Split into train and test

train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [ ]:
#Tokenizing the dataset

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
def tokenize_function(examples):
  return tokenizer(examples["text"],padding="max_length",truncation=True)

In [ ]:
tokenized_train = train_dataset.map(tokenize_function,batched=True)
tokenized_test = test_dataset.map(tokenize_function,batched=True)

#Pytorch tensors

tokenized_train.set_format("torch",columns=["input_ids","attention_mask","label"])
tokenized_test.set_format("torch",columns=["input_ids","attention_mask","label"])

tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_test = tokenized_test.rename_column("label", "labels")

train_dataloader = DataLoader(tokenized_train,shuffle=True,batch_size=8)
eval_dataloader = DataLoader(tokenized_test,batch_size=8)

# **Defining Encoder Models**

In [ ]:
model_names = ['bert-base-cased','distilbert-base-cased','roberta-base']

models = [AutoModelForSequenceClassification.from_pretrained(name,num_labels=2) for name in model_names]

# **Training and Evaluation**

In [ ]:
def train_epoch(model,dataloader,optimizer,lr_scheduler, device):
  model.train()
  total_loss = 0
  for batch in dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    total_loss += loss.item()

    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()

  return total_loss/len(dataloader)

In [ ]:
def evaluate(model, dataloader, device):
  model.eval()
  total_loss=0
  correct=0
  total=0

  with torch.no_grad():
    for batch in dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      loss = outputs.loss
      total_loss += loss.item()

      predictions = outputs.logits.argmax(dim=-1)
      correct += (predictions == batch["labels"]).sum().item()
      total += len(batch["labels"])

    return total_loss/len(dataloader), correct/total

# **Training the Models**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 3

for model in models:
  model.to(device)

  optimizer = AdamW(model.parameters(),lr=5e-5)
  num_training_steps = num_epochs * len(train_dataloader)
  lr_scheduler = get_scheduler(
      name="linear",optimizer=optimizer,num_warmup_steps=0,num_training_steps=num_training_steps
  )

  for epoch in range(num_epochs):
    train_loss = train_epoch(model,train_dataloader,optimizer,lr_scheduler,device)
    test_loss, test_accuracy = evaluate(model, eval_dataloader,device)

    print(f"Epoch {epoch+1}")
    print(f"Train Loss: {train_loss: .4f}")
    print(f"Test Loss: {test_loss: .4f}")
    print(f"Test Accuracy: {test_accuracy: .4f}")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1
Train Loss:  0.1916
Test Loss:  0.2119
Test Accuracy:  0.9192
